### Import components

In [ ]:
from src.document_loader import load_pdf_document
from src.chunking import chunk_text
from src.embedding import Embedder
from src.vector_store import FaissVectorStore
from src.prompt_constructor import construct_prompt
from src.lm import LM

## Offline Preparation

In [ ]:
PDF_DOCUMENT_PATH = r"./data/document.pdf"

In [ ]:
LOADED_DOCUMENT_PATH = r"./data/document.md"

### Load document(s)

In [ ]:
pages = load_pdf_document(PDF_DOCUMENT_PATH, LOADED_DOCUMENT_PATH)

### Chunk document(s)

In [ ]:
chunks = chunk_text(LOADED_DOCUMENT_PATH)

### Embed document(s) chunks

Please install the following package:
```bash
pip install -U sentence-transformers
```

In [ ]:
embedder = Embedder()

In [ ]:
embedded_chunks = embedder.embed_chunks(chunks)

In [ ]:
embeddings = [chunk["embedding"] for chunk in embedded_chunks]

### Store vectors of embedded document(s) into a vector index

In [ ]:
faissVectorStore = FaissVectorStore(len(embeddings[0]))

In [ ]:
faissVectorStore.add(embeddings)

## Online Phase

### Embed user query

In [ ]:
query = "What is Advanced RAG?"

In [ ]:
query_embedding = embedder.embed_query(query)

### Retrieve relevant chunks from the stored vector index

In [ ]:
retrieved_chuncks = faissVectorStore.search(query_embedding)

### Construct prompt

In [ ]:
prompt = construct_prompt(query, retrieved_chuncks, embedded_chunks)

### Generate answer with SLM

Please run the following commands in your terminal to access to huggingface models.

```bash
pip install huggingface_hub
```

After installing its package, please provide a token to login. 

You can create a token from [huggingface.co/settings/tokens](https://huggingface.co/settings/tokens)

```bash
huggingface-cli login
```

In [ ]:
lm = LM()

Without RAG:

In [ ]:
output = lm.generate_text(query)
print(output)

With RAG:

In [ ]:
output = lm.generate_text(prompt)
print(output)